In [1]:
import pandas as pd

In [2]:
class YiLong:



    def __init__(self, type):
        
        # check correct input
        assert type == 'Classification' or type == 'Regression'

        self.clf_type = type
        self._initialise_objects() # Initialise objects
        print(f'YiLong Initialised to analyse {self.clf_type}')



    def _initialise_objects(self):
        """ Helper to initialise objects """

        self.regression_extra_output_columns = ['Train r2', 'Val r2', 'Test r2', 
            'Train RMSE', 'Val RMSE', 'Test RMSE', 'Train MAPE', 'Val MAPE', 'Test MAPE', 'Time']
        self.classification_extra_output_columns = ['Train accu', 'Val accu', 'Test accu', 
            'Train balanced_accu', 'Val balanced_accu', 'Test balanced_accu', 'Train f1', 'Val f1', 'Test f1', 
            'Train precision', 'Val precision', 'Test precision', 'Train recall', 'Val recall', 'Test recall', 'Time']
        self.tuning_result = None
        self.hyperparameters = None

    

    def read_tuning_result(self, address):
        """ Read in Tuning Result """

        self.tuning_result = pd.read_csv(address)

        print(f'Successfully read in tuning result, with {len(self.tuning_result)} columns')

        # get list of hyperparameters by taking what is not in the extra_output_columns
        if self.clf_type == 'Classification':
            self.hyperparameters = [col for col in self.tuning_result.columns if col not in self.classification_extra_output_columns]

        elif self.clf_type == 'Regression':
            self.hyperparameters = [col for col in self.tuning_result.columns if col not in self.regression_extra_output_columns]



    def read_sorted_full_df(self):
        """ View dataframe sorted in reverse in terms of validation score """

        if self.clf_type =='Regression':
            display(self.tuning_result.sort_values(['Val r2'], ascending = False))
        elif self.clf_type =='Classification':
            display(self.tuning_result.sort_values(['Val accu'], ascending = False))

    

    def read_mean_val_scores(self):
        """ View the means of evaluation metrics for combinations containing each individual value of a hyperparameter – for each hyperparameter """

        for col in self.hyperparameters: # for each hyperparameter

            print(col)

            hyperparameter_values = list(set(self.tuning_result[col]))
            hyperparameter_values.sort()
            
            # create this temporary dataframe
            validation_score_df = pd.DataFrame()
            for value in hyperparameter_values: # for each value in the hyperparameter
                tmp_df = self.tuning_result[self.tuning_result[col] == value] # select df with only those parameter values

                # get means
                if self.clf_type == 'Classification':
                    tmp_df_mean = tmp_df[self.classification_extra_output_columns[:-1]].mean().T
                elif self.clf_type == 'Regression':
                    tmp_df_mean = tmp_df[self.regression_extra_output_columns[:-1]].mean().T

                # get number of observations in this group
                tmp_df_mean['n'] = len(tmp_df)

                # append to this temporary dataframe
                validation_score_df[f'{value}'] = tmp_df_mean

            display(validation_score_df)

In [3]:
yilong = YiLong('Regression')

YiLong Initialised to analyse Regression


In [36]:
yilong.read_tuning_result('../models/tuning/N_xgb_1.csv')

Successfully read in tuning result, with 1350 columns


In [37]:
yilong.read_sorted_full_df()

,gamma,subsample,colsample_bytree,max_depth,eta,Train r2,Val r2,Test r2,Train RMSE,Val RMSE,Test RMSE,Train MAPE,Val MAPE,Test MAPE,Time
1174,0.01,0.75,0.75,5.0,0.30,0.6385,0.4958,0.4626,0.1513,0.1787,0.1846,1.236698e+14,1.510008e+14,1.546304e+14,12.71
482,0.01,0.75,0.75,5.0,0.15,0.6385,0.4958,0.4626,0.1513,0.1787,0.1846,1.236698e+14,1.510008e+14,1.546304e+14,12.62
619,0.01,0.75,0.75,5.0,0.45,0.6385,0.4958,0.4626,0.1513,0.1787,0.1846,1.236698e+14,1.510008e+14,1.546304e+14,12.36
226,0.01,0.75,0.75,5.0,0.75,0.6385,0.4958,0.4626,0.1513,0.1787,0.1846,1.236698e+14,1.510008e+14,1.546304e+14,13.79
1254,0.01,0.75,0.75,5.0,0.60,0.6385,0.4958,0.4626,0.1513,0.1787,0.1846,1.236698e+14,1.510008e+14,1.546304e+14,12.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,100.00,0.25,0.50,50.0,0.60,0.1006,0.1011,0.1003,0.2387,0.2386,0.2388,2.687080e+14,2.686933e+14,2.690300e+14,29.61
90,100.00,0.25,0.50,10.0,0.45,0.1006,0.1011,0.1003,0.2387,0.2386,0.2388,2.687080e+14,2.686933e+14,2.690300e+14,15.99
84,100.00,0.25,0.50,10.0,0.75,0.1006,0.1011,0.1003,0.2387,0.2386,0.2388,2.687080e+14,2.686933e+14,2.690300e+14,15.96
585,100.00,0.25,0.50,50.0,0.15,0.1006,0.1011,0.1003,0.2387,0.2386,0.2388,2.687080e+14,2.686933e+14,2.690300e+14,29.78


In [64]:
yilong.read_mean_val_scores()

n_neighbors


,5.0,10.0,15.0,20.0,25.0,30.0,35.0,40.0,45.0,50.0,...,155.0,160.0,165.0,170.0,175.0,180.0,185.0,190.0,195.0,200.0
Train r2,7.442000e-01,7.136000e-01,7.032000e-01,6.961000e-01,6.926000e-01,6.897500e-01,6.876500e-01,6.858000e-01,6.846500e-01,6.834000e-01,...,6.696500e-01,6.691500e-01,6.687500e-01,6.684000e-01,6.681500e-01,6.678000e-01,6.677000e-01,6.673500e-01,6.671000e-01,6.668500e-01
Val r2,2.422000e-01,2.985500e-01,3.171000e-01,3.315500e-01,3.350000e-01,3.365000e-01,3.398500e-01,3.383500e-01,3.391000e-01,3.387500e-01,...,3.320000e-01,3.320000e-01,3.318000e-01,3.313500e-01,3.304500e-01,3.301500e-01,3.295500e-01,3.288000e-01,3.285000e-01,3.289500e-01
Test r2,2.271000e-01,2.855500e-01,3.074000e-01,3.129000e-01,3.153000e-01,3.194500e-01,3.218500e-01,3.244500e-01,3.243500e-01,3.268000e-01,...,3.286500e-01,3.281500e-01,3.279500e-01,3.276000e-01,3.273500e-01,3.267000e-01,3.258500e-01,3.259500e-01,3.257500e-01,3.253500e-01
Train RMSE,9.000000e-02,9.525000e-02,9.695000e-02,9.810000e-02,9.865000e-02,9.910000e-02,9.945000e-02,9.975000e-02,9.990000e-02,1.001000e-01,...,1.022500e-01,1.023500e-01,1.024000e-01,1.024500e-01,1.025000e-01,1.025500e-01,1.025500e-01,1.026500e-01,1.026500e-01,1.027000e-01
Val RMSE,2.190500e-01,2.107500e-01,2.079500e-01,2.057500e-01,2.052000e-01,2.050000e-01,2.044500e-01,2.047000e-01,2.046000e-01,2.046000e-01,...,2.056500e-01,2.056500e-01,2.057000e-01,2.057500e-01,2.059000e-01,2.059500e-01,2.060500e-01,2.061500e-01,2.062000e-01,2.061500e-01
Test RMSE,2.213000e-01,2.128000e-01,2.095000e-01,2.086500e-01,2.083500e-01,2.077000e-01,2.073500e-01,2.069500e-01,2.069500e-01,2.065500e-01,...,2.063000e-01,2.063500e-01,2.064000e-01,2.064500e-01,2.065000e-01,2.066000e-01,2.067000e-01,2.067000e-01,2.067000e-01,2.068000e-01
Train MAPE,7.652407e+13,8.674543e+13,9.006546e+13,9.240581e+13,9.387751e+13,9.498564e+13,9.580827e+13,9.650415e+13,9.701758e+13,9.745227e+13,...,1.023961e+14,1.025476e+14,1.026651e+14,1.028042e+14,1.029009e+14,1.030024e+14,1.031023e+14,1.032227e+14,1.033358e+14,1.034343e+14
Val MAPE,1.934418e+14,1.951865e+14,1.962055e+14,1.962319e+14,1.975670e+14,1.984823e+14,1.989540e+14,1.999540e+14,2.006749e+14,2.013337e+14,...,2.075370e+14,2.077030e+14,2.078236e+14,2.080855e+14,2.083919e+14,2.085528e+14,2.087310e+14,2.089513e+14,2.090993e+14,2.091356e+14
Test MAPE,1.946311e+14,1.959961e+14,1.967437e+14,1.993802e+14,2.005412e+14,2.010231e+14,2.016341e+14,2.015736e+14,2.022933e+14,2.025083e+14,...,2.074638e+14,2.076508e+14,2.078048e+14,2.080210e+14,2.081653e+14,2.084474e+14,2.087008e+14,2.088030e+14,2.089126e+14,2.090811e+14
n,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,...,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00


weights


,distance,uniform
Train r2,1.000000e+00,3.576725e-01
Val r2,3.314350e-01,3.299975e-01
Test r2,3.234375e-01,3.227125e-01
Train RMSE,0.000000e+00,2.016225e-01
Val RMSE,2.057300e-01,2.059675e-01
Test RMSE,2.070675e-01,2.071825e-01
Train MAPE,0.000000e+00,1.981142e+14
Val MAPE,2.038596e+14,2.043837e+14
Test MAPE,2.043674e+14,2.047992e+14
n,4.000000e+01,4.000000e+01
